In [ ]:
import os
import random
import torch
import numpy as np 
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torchvision import transforms
from tqdm.notebook import tqdm
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.kid import KernelInceptionDistance
from torchmetrics.image.mifid import MemorizationInformedFrechetInceptionDistance

from dataset import init_dataset

In [ ]:
experiment_dir = "/datatmp/users/igeorvasilis/ddpm-continual-learning/results/mnist/exp_mask/"
args_dir = os.path.join(experiment_dir, "args.pt")
args = torch.load(args_dir)

args.pretrained_model_dir = os.path.join(experiment_dir, "epoch-100")
args.train_log_dir = os.path.join(experiment_dir, "train_log.pt")
args.folder_name = "ddim_m_fake_images"

train_log = torch.load(args.train_log_dir)

torch.manual_seed(args.gen_seed)
np.random.seed(args.gen_seed)
random.seed(args.gen_seed)

In [ ]:
# Define the preprocessing function
preprocess = None

if hasattr(args, 'preprocess'):
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
        transforms.Resize(32, antialias=True),
    ])

# Evaluation

### Frechet Inception Distance (FID)

In [ ]:
# List all fake images from the corresponding directory 
fake_images_dir = f'{args.pretrained_model_dir}/{args.folder_name}'
fake_images_list = os.listdir(fake_images_dir)
n_fake_images = len(fake_images_list)

# Create eval dataloader
trainset, testset = init_dataset(dataset_name=args.dataset_name, dataset_path=args.dataset_path, labels=args.labels, preprocess=preprocess)
trainset = torch.utils.data.ConcatDataset([trainset, testset])
eval_dataloader = torch.utils.data.DataLoader(
    trainset, batch_size=args.eval_batch_size, shuffle=True)

# Define FID metric
fid = FrechetInceptionDistance(feature=2048, normalize=True).to(args.device)

# Iterate over all fake images
n_images_to_eval = min(len(fake_images_list), len(eval_dataloader.dataset))
for batch_idx in tqdm(range(0, n_images_to_eval, args.eval_batch_size), 
                      desc=f'Calculating FID for {n_images_to_eval} images...'):
    
    # Get the real images
    real_images, _ = next(iter(eval_dataloader))
    real_images = real_images.to(args.device)

    # Get the fake images
    fake_images = [read_image(f"{fake_images_dir}/{i}") for i in fake_images_list[batch_idx:batch_idx+args.eval_batch_size]]
    fake_images = torch.stack(fake_images).to(args.device) 
    fake_images = fake_images.float() / 255.0
    
    real_images = real_images[:fake_images.shape[0]]

    # Update the FID metric
    fid.update(real_images, real=True)
    fid.update(fake_images, real=False)

# Compute the FID score
print(fid.compute())
fid.reset()

### Kernel Inception Distance (KID)

In [ ]:
# List all fake images from the corresponding directory 
fake_images_dir = f'{args.pretrained_model_dir}/{args.folder_name}'
fake_images_list = os.listdir(fake_images_dir)
n_fake_images = len(fake_images_list)

# Create eval dataloader
trainset, testset = init_dataset(dataset_name=args.dataset_name, dataset_path=args.dataset_path, labels=args.labels, preprocess=None)
trainset = torch.utils.data.ConcatDataset([trainset, testset])
eval_dataloader = torch.utils.data.DataLoader(
    trainset, batch_size=args.eval_batch_size, shuffle=True)

# Define kid metric
subsets = 100
subset_size = 1000
kid = KernelInceptionDistance(
    feature=2048, 
    normalize=True,
    subsets=subsets,
    subset_size=subset_size
).to(args.device)

# Iterate over all fake images
n_images_to_eval = min(len(fake_images_list), len(eval_dataloader.dataset))
for batch_idx in tqdm(range(0, n_images_to_eval, args.eval_batch_size), desc='Calculating kid...'):
    
    # Get the corresponding real images
    real_images, _ = next(iter(eval_dataloader))
    real_images = real_images.to(args.device)

    # Get the fake images
    fake_images = [read_image(f"{fake_images_dir}/{i}") for i in fake_images_list[batch_idx:batch_idx+args.eval_batch_size]]
    fake_images = torch.stack(fake_images).to(args.device) 
    fake_images = fake_images.float() / 255.0

    # Update the kid metric
    kid.update(real_images, real=True)
    kid.update(fake_images, real=False)

# Compute the kid score
print(kid.compute())
kid.reset()

### MFID

In [ ]:
# List all fake images from the corresponding directory 
fake_images_dir = f'{args.pretrained_model_dir}/{args.folder_name}'
fake_images_list = os.listdir(fake_images_dir)
n_fake_images = len(fake_images_list)

# Create eval dataloader
trainset, testset = init_dataset(dataset_name=args.dataset_name, dataset_path=args.dataset_path, labels=args.labels, preprocess=preprocess)
trainset = torch.utils.data.ConcatDataset([trainset, testset])
eval_dataloader = torch.utils.data.DataLoader(
    trainset, batch_size=args.eval_batch_size, shuffle=True)

# Define FID metric
mifid = MemorizationInformedFrechetInceptionDistance(feature=2048, normalize=True).to(args.device)

# Iterate over all fake images
n_images_to_eval = min(len(fake_images_list), len(eval_dataloader.dataset))
for batch_idx in tqdm(range(0, n_images_to_eval, args.eval_batch_size), 
                      desc=f'Calculating mifid for {n_images_to_eval} images...'):
    
    # Get the real images
    real_images, _ = next(iter(eval_dataloader))
    real_images = real_images.to(args.device)

    # Get the fake images
    fake_images = [read_image(f"{fake_images_dir}/{i}") for i in fake_images_list[batch_idx:batch_idx+args.eval_batch_size]]
    fake_images = torch.stack(fake_images).to(args.device) 
    fake_images = fake_images.float() / 255.0
    
    # real_images = real_images[:fake_images.shape[0]]

    # Update the mifid metric
    mifid.update(real_images, real=True)
    mifid.update(fake_images, real=False)

# Compute the mifid score
print(mifid.compute())
mifid.reset()

### Precision Recall

In [ ]:
trainset, _ = init_dataset(dataset_name=args.dataset_name, dataset_path=args.dataset_path, labels=args.labels)
eval_dataloader = torch.utils.data.DataLoader(
    trainset, batch_size=args.eval_batch_size, shuffle=True)

real_images = torch.cat([image for image, _ in eval_dataloader])

In [ ]:
# List all fake images from the corresponding directory 
fake_images_dir = f'{args.pretrained_model_dir}/{args.folder_name}'
fake_images_list = os.listdir(fake_images_dir)
fake_images = torch.stack([read_image(f'{fake_images_dir}/{i}') for i in fake_images_list])
fake_images = fake_images.float() / 255.0

In [ ]:
n_images_to_eval = min(real_images.shape[0], fake_images.shape[0])
real_images = real_images[:n_images_to_eval]
fake_images = fake_images[:n_images_to_eval]

In [ ]:
from improved_precision_recall import IPR 

# Define IPR metric
ipr = IPR(batch_size=8, k=3, num_samples=n_images_to_eval, device='cuda')

# Compute Manifold 
ipr.compute_manifold_ref(real_images)

In [ ]:
metric = ipr.precision_and_recall(fake_images)
# Print results
print('precision =', metric.precision)
print('recall =', metric.recall)

# r_score = ipr.realism(fake_images)
# print('realism =', r_score)

### Classifier

In [ ]:
from torchvision import models
import torch.nn as nn

checkpoint = torch.load('./results/eval_classifier/checkpoint_2n7.pth')

# Load the actual and the opposite mapping
map_labels = checkpoint['mapping']
map_labels_inv = {v: k for k, v in map_labels.items()}

# Define the pretrained model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(args.labels))

model.load_state_dict(checkpoint['state_dict'])
model = model.to(args.device)

In [ ]:
# List all fake images from the corresponding directory 
fake_images_dir = f'./{args.pretrained_model_dir}/{args.folder_name}'
fake_images_list = os.listdir(fake_images_dir)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import math

n_counts = {label:0 for label in map_labels.keys()}

for b_idx in tqdm(range(0, len(fake_images_list), args.eval_batch_size)):
    
    fake_images_names = fake_images_list[b_idx:b_idx+args.eval_batch_size]
    fake_images = [ read_image(f'{fake_images_dir}/{fi_name}') for fi_name in fake_images_names ]
    fake_images = torch.stack(fake_images)
    fake_images = fake_images.float() / 255.0

    # Get predictions
    outputs = model(fake_images.to(args.device))
    _, predicted = torch.max(outputs, 1)
    predicted = [map_labels_inv[label.item()] for label in predicted]
    for label in predicted: n_counts[label] += 1

# Compute the total count of all labels
total_count = sum(n_counts.values())
# Compute the frequency of each label
frequencies = {label: count / total_count for label, count in n_counts.items()}
print("Frequencies:", frequencies)

# Compute the entropy
entropy = -sum([p * math.log(p) for p in frequencies.values()])
print("Entropy:", entropy)

### Visualize

In [ ]:
import matplotlib.pyplot as plt
from diffusers.utils import make_image_grid

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

fakes = [transforms.ToPILImage()(image) for image in fake_images[:32]]
grid = make_image_grid(fakes, rows=4, cols=8)
axes[0].imshow(grid); axes[0].set_title(f'Fake images')

reals = [transforms.ToPILImage()(image) for image in real_images[:32]]
grid = make_image_grid(reals, rows=4, cols=8)
axes[1].imshow(grid); axes[1].set_title(f'Real images')